In [1]:

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline


In [2]:
df=pd.read_csv(f'../data/raw/diamonds_train.csv')
dfpredict=pd.read_csv(f'../data/raw/diamonds_predict.csv')


In [3]:
df['df_original']=1
dfpredict['df_original']=0
print(df.shape)
print(dfpredict.shape)

df=pd.concat([df,dfpredict], axis=0)
df['df_original'].value_counts()


(40455, 11)
(13485, 11)


1    40455
0    13485
Name: df_original, dtype: int64

In [4]:
print(df.info(memory_usage='deep'))
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53940 entries, 0 to 13484
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   carat        53940 non-null  float64
 1   cut          53940 non-null  object 
 2   color        53940 non-null  object 
 3   clarity      53940 non-null  object 
 4   depth        53940 non-null  float64
 5   table        53940 non-null  float64
 6   price        40455 non-null  float64
 7   x            53940 non-null  float64
 8   y            53940 non-null  float64
 9   z            53940 non-null  float64
 10  df_original  53940 non-null  int64  
 11  id           13485 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 13.9 MB
None


,carat,depth,table,price,x,y,z,df_original,id
count,53940.000000,53940.000000,53940.000000,40455.000000,53940.000000,53940.000000,53940.000000,53940.000000,13485.000000
mean,0.797940,61.749405,57.457184,3928.444469,5.731157,5.734526,3.538734,0.750000,6742.000000
std,0.474011,1.432621,2.234491,3992.416147,1.121761,1.142135,0.705699,0.433017,3892.928525
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,945.000000,4.710000,4.720000,2.910000,0.750000,3371.000000
50%,0.700000,61.800000,57.000000,2397.000000,5.700000,5.710000,3.530000,1.000000,6742.000000
75%,1.040000,62.500000,59.000000,5331.000000,6.540000,6.540000,4.040000,1.000000,10113.000000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000,1.000000,13484.000000


In [5]:
df['cut'] = df['cut'].astype('category')
df['color'] = df['color'].astype('category')
df['clarity'] = df['clarity'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53940 entries, 0 to 13484
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   carat        53940 non-null  float64 
 1   cut          53940 non-null  category
 2   color        53940 non-null  category
 3   clarity      53940 non-null  category
 4   depth        53940 non-null  float64 
 5   table        53940 non-null  float64 
 6   price        40455 non-null  float64 
 7   x            53940 non-null  float64 
 8   y            53940 non-null  float64 
 9   z            53940 non-null  float64 
 10  df_original  53940 non-null  int64   
 11  id           13485 non-null  float64 
dtypes: category(3), float64(8), int64(1)
memory usage: 4.3 MB


In [6]:
df['cut_int']=''

df.loc[df['cut']=='Fair','cut_int']=1
df.loc[df['cut']=='Good','cut_int']=2
df.loc[df['cut']=='Very Good','cut_int']=3
df.loc[df['cut']=='Premium','cut_int']=4
df.loc[df['cut']=='Ideal','cut_int']=5

df['cut_int'] = df['cut_int'].astype('int64')

df['color_int']=''

df.loc[df['color']=='J','color_int']=1
df.loc[df['color']=='I','color_int']=2
df.loc[df['color']=='H','color_int']=3
df.loc[df['color']=='G','color_int']=4
df.loc[df['color']=='F','color_int']=5
df.loc[df['color']=='E','color_int']=6
df.loc[df['color']=='D','color_int']=7


df['color_int'] = df['color_int'].astype('int64')

df['clarity_int']=''

df.loc[df['clarity']=='I1','clarity_int']=1
df.loc[df['clarity']=='SI2','clarity_int']=2
df.loc[df['clarity']=='SI1','clarity_int']=3
df.loc[df['clarity']=='VS2','clarity_int']=4
df.loc[df['clarity']=='VS1','clarity_int']=5
df.loc[df['clarity']=='VVS2','clarity_int']=6
df.loc[df['clarity']=='VVS1','clarity_int']=7
df.loc[df['clarity']=='IF','clarity_int']=8


df['clarity_int'] = df['clarity_int'].astype('int64')

In [7]:
df['cut_color_clarity_score'] = df.apply(lambda new: 3*new['cut_int']+3*new['color_int']+4*new['clarity_int'], axis=1)

In [8]:
df['raw_density'] = (df['carat']*0.2)/(df['x']*df['y']*df['z']/3/1000)
df['raw_density'] = np.where(df['raw_density']==float('Inf'),0,df['raw_density'])


In [9]:
def marketing_carat(x):
    h=0.09
    for z in range(1,6):
        if (z-h)<=x<(z):             #bad 0.91 a 0.99  good  1.00 a 1.04
            y='bad'
            break
        elif (z)<=x<=(z+h-0.05):
            y='good'
            break
        else:
            y='regular'
    return y

df['marketing_carat']=df.apply(lambda x: marketing_carat(x['carat']), axis=1) 
    
df['marketing_carat_int']=''

df.loc[df['marketing_carat']=='bad','marketing_carat_int']=1
df.loc[df['marketing_carat']=='regular','marketing_carat_int']=2
df.loc[df['marketing_carat']=='good','marketing_carat_int']=3



df['marketing_carat_int'] = df['marketing_carat_int'].astype('int64')    
df['marketing_carat'].value_counts()


regular    45820
good        6800
bad         1320
Name: marketing_carat, dtype: int64

In [10]:
df['carat_sqr']=df['carat']**4

# df['carat_sqr'] = np.where(df['carat_sqr']==float('Inf'),0,df['carat_sqr'])

In [11]:

# sns.set(font_scale=1)
# dft=df.corr(method='spearman')
# mask = np.triu(np.ones_like(dft, dtype=np.bool))
# f, ax = plt.subplots(figsize=(20, 10))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(dft, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

# plt.show()

In [12]:
dfd=pd.get_dummies(df)

In [13]:
df.to_parquet('../data/processed/df.parquet')
dfd.to_parquet('../data/processed/dfd.parquet')

In [14]:
df['df_original'].value_counts()

1    40455
0    13485
Name: df_original, dtype: int64